<a href="https://colab.research.google.com/github/JacobFV/Nodal-RL/blob/master/notebooks/Nodal_RL_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import tensorflow as tf

In [17]:
tf.version.VERSION, tf.version.COMPILER_VERSION, tf.version.GRAPH_DEF_VERSION

('2.2.0', '5.4.0', 175)

In [2]:
import os
import warnings

In [3]:
class Node:
    """
    Generic Node base class
    """
    
    def __init__(self,
                 p = 1,
                 default_parent_biasing_params = {
                     "w_otarget": 1.0,
                     "o_target_lambda": 1.0
                 }):
        """
        initializese abstract node class
        
        :param p: period for timesteps. number
        :param default_parent_biasing_params:
                default biasing_params for parents
                Dict<str,number>
                
                properties:
                    - otarget weight: "w_otarget" -> number
                    - recency otarget weight decay factor: "o_target_lambda" -> number
        
        :return: returns Node
        """
        
        self.p = p
        
        self.default_parent_biasing_params = default_parent_biasing_params
        
        """
        data for each parent node
        Dict<Node, Dict<str, number>>

        usage: [Node]["property"] = value

        properties:
            - otarget weight: 'w_otarget' -> number
            - recency otarget weight decay factor: 'o_target_lambda' -> number
        """
        self.parent_biasing_params = {}
        
        #initialize stateful variables
        self.t = 0
        
        
        """
        actual observations used from parents

        NOTE: the parent may have more observations
            in its `s` records that are not used by
            `self` because they are at a finer timescale 

        dict<Node, dict<number,target observation>>
        """
        self.parent_o = {}

        """
        biasing target observations delivered to parents

        dict<Node, dict<number,target observation>>
        """
        self.parent_otarget = {}

        """
        top down biasing target observations obeyed from children

        NOTE: the children may have more target observations
            in their `parent_otarget` records that are not used by
            `self` because they are at a finer timescale 

        dict<Node, dict<number,target observation>>
        """
        self.child_otarget = {}
        
        """
        record of states for each timestep
        that `update` is called.

        Some states are referenced by children

        Dict<number, state>
        """
        self.s = {}
    
    def __call__(self, *parents):
        """
        register `parents` as its own parent and reach into
        `parents` to register as child. Since this method simply
        adds parents, it can be called multiple times with subsets
        of parents or once with all parents to achieve equivalent
        effects. Applies `self.default_parent_biasing_params` to
        all parents. To manually specify `parent_biasing_params`,
        pass in `parents` as dict<Node, parent_biasing_params>
        
        :param *parents: list of parent nodes to connect
                or dict<Node, parent_biasing_params>
        
        :return: returns nothing
        """

        #convert parents to dict<Node, parent_biasing_params>
        #regardless of how presented in args
        if(isinstance(parents, list)):
            parents = {
                parent, self.default_parent_biasing_params
                for parent in parents}
            
        for parent, parent_biasing_params in parents.items():
            self._add_parent(parent, parent_biasing_params)
            parent._add_child(self)
    
    def _add_parent(self, parent, parent_biasing_params):
        self.parent_biasing_params[parent] = parent_biasing_params
        self.parent_o[parent] = {}
        self.parent_otarget[parent] = {}
    
    def _add_child(self, child):
        self.child_otarget[child] = {}

    def update(self, time):
        """
        update node's internal variables at time intervals `self.p`
        This function should be overriden 
        
        :param time: node time when updated

        :return: returns nothing
        """
        raise NotImplemented()

    def closest_record(self, dictionary, time):
        """
        Identify highest indexed record in `dictionary` 
        that is less than or equal to `time`
        
        NOTE: call from Node containing dictionary.
        because later vesions of this algorithm may use
        `self.p` to jump ahead intelligently
        
        :param dictionary: dict<number, obj> to query
        :param time: time to query for closest entries to
        
        :return: returns closest matching (time, record) record
        """
        
        for tau in dictionary.keys().reverse():
            if tau <= time:
                return tau, dictionary[tau]
        
        #errors
        if not bool(dictionary):
            raise Exception("`dictionary` empty")
        raise Exception("error searching for closest record to time:{time} in `dictionary`")
            
    def child_obs_target_weighted_mean(self, other_terms=[]):
        """
        computes weighted mean of child nodes'
        target observations for `self` and `other_terms`
        
        The weighted mean observation target is
        used by actuators and information nodes
        
        In computing `child_obs_target_weighted_mean`,
        child node target observations are selected
        by recency and added to `self.child_otarget`
        for later training
        
        :param other_terms: (opt.) [(weight, value)] tuple list
                of additional terms to incorperate into the
                weighted mean calculation

        :return: returns top down biasing target state  
        """
        
        weights = [weight for weight, value in other_terms]
        values = [value for weight, value in other_terms]
        
        # for each child
        for child, biasing_params in self.parent_biasing_params.items():
            # find child target observation for time tau
            # that is closest to `self.t`
            tau, otarget = child.closest_record(child.s, self.t + self.p)
            values.append(otarget)
            
            # compute weight in mean
            weights.append(
                biasing_params["w_otarget"]
                * CER(otarget)
                * np.exp(
                    -biasing_params["o_target_lambda"]
                    *(self.t + self.p - tau)
                )
            )
            
            # save otarget for later training 
            self.child_otarget[child][tau] = otarget
        
        # compute weighted sum
        weighted_sum = sum([
            weight * value
            for weight, value
            in zip(weights, values)
        ])
        
        # return weighted mean
        return weighted_sum / sum(weights)

    def save_information_episode(self, path):
        """
        saves `self.parent_o` and `self.child_otarget`
        to `path` for training later.
        
        This does NOT include subclass definitions, hyperparameters, etc.
        Those entities must be reconstructed and then `load_information_episode`
        
        :param path: savepath
                
                The suffix of `path` detirmines decompression
                employed. Valid options are: '.zip', '.gz', and '.bz2'
                
                `path` must use existing directories
        
        :return: returns nothing
        """
        
        datasets = {} # {'filename': tf.data.Dataset}
        for parent, observations in self.parent_o.items():
            # generate dataset and add to `datasets`
            pass
        
        for parent, observations in self.child_otarget.items():
            # generate dataset and add to `datasets`
            pass
        
        # get compression format
        root_path, extension = os.path.splitext(path)
        
        # make uncompressed dir
        head, tail = os.path.split(path)
        # TODO: create dir at `head` named `tail`
        
        # collectively save all datasets
        for name, dataset in datasets.items():
            #TODO: save compressed `dataset` at `os.path.join(root_path,name)`
        
        # finally, compress the containing dir
        # TODO select appropriate compression algorithm
        # TODO actually compress and save dir with pathname `path`
        
        raise NotImplemented()
        
    def load_information_episode(self, path):
        """
        load data to `self.parent_o` and `self.child_otarget`
        from `path` for training. Keeps existing data, but
        override records when found in file at `path`
        
        NOTE: `self.parent_otarget` and `self.s` are
        not loaded by this call. To ensure a completely new
        information episode is loaded, you should
        `reset_information_episode` prior to loading another.
        
        This does NOT include subclass definitions, hyperparameters, etc.
        Those entities must be reconstructed and then `load_information_episode`
        
        :param path: savepath
                
                The suffix of `path` detirmines decompression
                employed. Valid options are: '.zip', '.gz', and '.bz2'
                
                `path` must use existing directories
        
        :return: returns nothing
        """
        raise NotImplemented()

    def reset_information_episode(self):
        """
        resets all inference stateful variables
        including observations and internal states

        This means clearing:
            `self.child_otarget`,
            `self.parent otarget`,
            `self.parent_o`,
            and `self.s` dictionaries
            
        :return: returns nothing
        """
        self.child_otarget.clear()
        self.parent_otarget.clear()
        self.s.clear()
        self.parent_o.clear()

In [4]:
class Sensory_Node(Node):

    def __init__(self, sensing_func):
        """
        initialize `Sensory_Node`
        
        :param sensing_func: function (time) -> observation
                used to interface with external environments
                
                use decorators to succintly initialize
                
        :param rand_obs: default `False`. whether or not `sensing_func`
                returns a 2D tensor (the second axis represents random
                variational values)
                
        :return: initialized `Sensory_Node`
        """
        self.sensing_func = sensing_func
        self.rand_obs = rand_obs
    
    def update(self, time):
        """
        records sensory observation in `self.s[time]`.
        Sensory observation taken from `self.sensing_func`
        
        :param time: node time when updated

        :return: returns nothing
        """
        self.s[time] = self.sensing_func(time)

In [5]:
class Actuator_Node(Node):

    def __init__(self,
                 acting_func,
                 sensing_func=None,
                 samples=1,
                 rand_obs=False,
                 **kwargs):
        """
        initialize `Sensory_Node`
        
        :param acting_func: function (time, target_observation) -> None
                used to interface with external environments
                
                use decorators to succintly initialize
        
        :param sensing_func: function (time) -> observation
                used to interface with external environments
                
                If `None`, target state is assumed to be
                executed perfectly
                
        :param samples: since the weighted mean child target 
                observation states may be probablistic, they are sampled
                for `self.samples` times to compute mean expectations.
                set to `-1` to directly pass all samples to acting func
                
        :param rand_obs: default `False`. whether or not `sensing_func`
                returns a 2D tensor (the second axis represents random
                variational values)
                
        :return: initialized `Actuator_Node`
        """
        
        if sensing_func is None:
            if rand_obs:
                warnings.warn('There will be no random variation'
                              + ' in percieved actions since'
                              + ' default perfect execution is assumed')
            def default_sensing_func(time)
            sensing_func
            
        self.acting_func = acting_func
        self.sensing_func = sensing_func
        self.samples = samples
        self.rand_obs = rand_obs
        
        super(Actuator_Node, self).__init__(**kwargs)
        
    def update(self, time):
        """
        attempts to act out the weighted mean child target observation.
        Since target states may be probablistic, they are sampled
        for `self.samples` times to compute mean expectations
        
        :param time: node time when updated

        :return: returns nothing
        """
        # record actual action perform at previous timestep
        if time - self.p >= 0:
            self.s.append(self.sensing_func(time))
        
        # compute target state
        starget = self.child_obs_target_weighted_mean(time)
        
        # sample target state
        if self.samples != -1:
            starget = tf.reduce_mean(starget, axis=1)
        
        # act out target state
        self.acting_func(starget)

In [9]:
class Information_Node(Node):

    
    def __init__(self, 
                 abstractor,
                 predictor,
                 policy,
                 params = {
                    "pred-weight":None,
                    "alpha-d1":None,
                    "alpha-d2":None,
                    "alpha-d3":None
                 },   
                 ):
        """
        initialize an `Information_Node`
        
        :return: returns initialized `Information_Node`
        """
        
        self.params = params
        self.abstractor
        self.predictor
        self.policy

    def update(self, time):
        """
        updates 
        
        :param time: node time when updated

        :return: returns nothing
        """
        #TODO
        
        # scramble values along random axis to increase randomness
        # TODO
        
        raise NotImplemented()

    def train(self,
              lr=0.001,
              verbose=True,
              beta_piD=1,
              beta_piC=1,
              beta_piI=1,
              beta_prD=1,
              beta_prC=1,
              ):
        """
        trains `Information_Node` from data
        stored in `self.parent_o` and `self.child_otarget`

        :param lr: learning rate for optimizer
        :param verbose: (True/False) log information verbosely
        :param beta_piD: training hyperparameter. See 'Putting It All Togethor'
        :param beta_piC: training hyperparameter. See 'Putting It All Togethor'
        :param beta_piI: training hyperparameter. See 'Putting It All Togethor'
        :param beta_prD: training hyperparameter. See 'Putting It All Togethor'
        :param beta_prC: training hyperparameter. See 'Putting It All Togethor'
        
        :return: returns nothing
        """
        #TODO
        raise NotImplemented()

In [ ]:
class Network:
    
    def __init__(self, nodes):
        """
        initializes the network
        
        :param nodes: all nodes that should be managed by the network. 
        
        :return: returns initialized network
        """
        self.nodes = nodes
        self.time = 0
    
    def run(self, duration=-1, timestep=1):
        """
        loops until time == duration
        
        :param duration: time to run for. -1 means forever
        :param timestep: time increments used during `step`
        
        :return: returns nothing
        """
        while self.time <= duration:
            self.step(timestep)
    
    def step(self, timestep):
        """
        increase timestep and check if any nodes should be updated
        
        :param timestep: positive time increment used
        
        :return: returns nothing
        """
        self.time += timestep
        #TODO
        raise NotImplemented()
    
    def save_episode(self, path):
        """
        saves stateful data to `path` in for training later.
        
        This does NOT save architecture, node class definitions,
        hyperparameters, etc. The network must be reconstructed
        and then records are added by `load_episode`
        
        :param path: filepath to compressed archive containing
                saved Node information_episodes
                
                The suffix of `path` detirmines compression
                employed. Valid options are: '.zip', '.gz', and '.bz2'
                
                `path` must use existing directories
        
        :return: returns nothing
        """
        #TODO
        raise NotImplemented()
    
    def load_episode(self, path):
        """
        Calls `load_information_episode` for all nodes
        with training records found at `path`. 
        
        NOTE: `Node.load_information_episode` has non-
        idempotent behavior. See definition for details
        
        This does NOT restore architecture, node class definitions,
        hyperparameters, etc. The network must be reconstructed
        and then records are added by `load_episode`
        
        :param path: filepath to compressed archive containing
                saved Node information_episodes
                
                The suffix of `path` detirmines decompression
                employed. Valid options are: '.zip', '.gz', and '.bz2'
                
                `path` must use existing directories
        
        :return: returns nothing
        """
        #TODO
        raise NotImplemented()
    
    def reset_episode(self):
        """
        resets all nodes' inference stateful variables
        including observations and internal states
            
        :return: returns nothing
        """
        #TODO
        raise NotImplemented()